# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup "a621a94c1be4036303d575da82f3b78c"
import hvplot.pandas
import pandas as pd
import requests
import hvplot.pandas
import hvplot as hv

# Import API key
from api_keys import geoapify_key

#geoapify_key = "api_keys.cpython-310"

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao vicente,-23.9631,-46.3919,23.57,66,0,0.75,BR,1727323687
1,1,beni mellal,32.3373,-6.3498,22.64,33,18,0.65,MA,1727323688
2,2,vadso,70.0744,29.7487,10.09,93,75,1.03,NO,1727323688
3,3,college,64.8569,-147.8028,6.99,87,75,2.57,US,1727323688
4,4,invercargill,-46.4000,168.3500,9.45,85,100,2.68,NZ,1727323416


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Initialize a map centered on a general location (latitude 20, longitude 0)
mymap = folium.Map(location=[20, 0], zoom_start=2)

# Create a marker cluster to group markers and avoid clutter
marker_cluster = MarkerCluster().add_to(mymap)

# Add city markers to the map
for index, row in city_data_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lon = row['Lng']
    humidity = row['Humidity']

    # Create a popup message with the city and humidity information
    popup_message = f"""
    City: {city}<br>
    Humidity: {humidity}%
    """

     # Add a marker to the map for each city with humidity info
    folium.Marker(
        location=[lat, lon],
        popup=popup_message,
        icon=folium.Icon(color="blue" if humidity < 60 else "red")
    ).add_to(marker_cluster)

# Save the map as an HTML file
mymap.save("output_data/humidity_map_folium.html")

# Display a message to confirm the map is saved
print("Map saved as 'output_data/humidity_map_folium.html'")

Map saved as 'output_data/humidity_map_folium.html'


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[city_data_df['Humidity'] < 60]    # humidity less than 60%

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,beni mellal,32.3373,-6.3498,22.64,33,18,0.65,MA,1727323688
36,36,jalai nur,49.4500,117.7000,17.64,39,94,7.48,CN,1727323695
40,40,klyuchi,52.2667,79.1667,7.30,53,22,3.75,RU,1727323696
43,43,jutai,-5.1833,-68.9000,26.64,55,99,1.36,BR,1727323696
47,47,fort st. john,56.2499,-120.8529,10.18,54,100,5.64,CA,1727323697


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,beni mellal,MA,32.3373,-6.3498,33,
36,jalai nur,CN,49.4500,117.7000,39,
40,klyuchi,RU,52.2667,79.1667,53,
43,jutai,BR,-5.1833,-68.9000,55,
47,fort st. john,CA,56.2499,-120.8529,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
geoapify_key

'3ad75bdd1e1d46a2a2c2ae895cf8ef74'

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": '',
    'bias': '',
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
beni mellal - nearest hotel: Tasmmet
jalai nur - nearest hotel: 满洲里福门客栈
klyuchi - nearest hotel: No hotel found
jutai - nearest hotel: No hotel found
fort st. john - nearest hotel: Fort St. John Motor Inn
margaret river - nearest hotel: Margaret River Hotel
ust'-kut - nearest hotel: Домашний очаг
queenstown - nearest hotel: Queens Hotel
bhawana - nearest hotel: No hotel found
nkowakowa - nearest hotel: Tzaneen Country Lodge
colchani - nearest hotel: Hotel de Sal - Sol de Mañana
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
heihe - nearest hotel: Санкт-Петербург
turkmenbasy - nearest hotel: Hazar Myhmanhanasy
connell - nearest hotel: No hotel found
bati - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
azangaro - nearest hotel: Hotel Princess Jairito
blackmans bay - nearest hotel: Villa Howden
bei'an - nearest hotel: No hotel found
tsetserleg - nearest hotel: Буян зочид буудал
mogocha - nearest hotel: 7Ka
roslavl' - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
1,beni mellal,MA,32.3373,-6.3498,33,Tasmmet
36,jalai nur,CN,49.4500,117.7000,39,满洲里福门客栈
40,klyuchi,RU,52.2667,79.1667,53,No hotel found
43,jutai,BR,-5.1833,-68.9000,55,No hotel found
47,fort st. john,CA,56.2499,-120.8529,54,Fort St. John Motor Inn
...,...,...,...,...,...,...
582,gannan,CN,47.9147,123.5031,36,No hotel found
585,paracatu,BR,-17.2222,-46.8747,37,Hotel Presidente
587,saurimo,AO,-9.6608,20.3916,39,Hotel Princesinha
590,qalat,AF,32.1058,66.9083,10,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, size=10, color='red', 
    tiles='OSM', hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)